# DNABert Training for G-quadruplex classification

---

Original used model: [armheb/DNA_bert_6](https://huggingface.co/armheb/DNA_bert_6?text=The+goal+of+life+is+%5BMASK%5D.)



## Load initial libraries, models, data:

In [1]:
!pip install transformers datasets --quiet

In [16]:
HF_ORIG_MODEL_NAME = 'armheb/DNA_bert_6'
# HF_MODEL_NAME = 'DNABert_K6_G_quad'  # trained for 25 epochs, overfits
HF_MODEL_NAME = 'DNABert_K6_G_quad_3'  # trained for 3 epochs

TMP_DIR = './q_quad_dnabert'
DATASET = 'roa7n/G_quad_DNA_tokenized_K6'

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

### Load model:

In [17]:
model = BertForSequenceClassification.from_pretrained(HF_ORIG_MODEL_NAME, num_labels=2)
model.to(device)

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--armheb--DNA_bert_6/snapshots/a79a8fd96ad172f964a4dbef3f4d7545a5034baa/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_ids": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_rnn_layer": 1,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "rnn": "lstm",
  "rnn_dropout": 0.0,
  "rnn_hidden": 768,
  "split": 10,
  "transformers_version": "4.22.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 4101
}

loading weights file pytorch_model.bin from cache at /home/jovyan/.cache/

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

## Get the data:

In [8]:
from datasets import load_dataset

dataset = load_dataset(DATASET)
dataset

Using custom data configuration roa7n--G_quad_DNA_tokenized_K6-226a4e1c505f34ab
Reusing dataset parquet (/home/jovyan/.cache/huggingface/datasets/roa7n___parquet/roa7n--G_quad_DNA_tokenized_K6-226a4e1c505f34ab/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['seq', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 300000
    })
    test: Dataset({
        features: ['seq', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 100000
    })
})

In [9]:
dataset['train'][0]['seq']

'TCATAGACCCGGTCTCATATGACAAGGAGGGGCATGTCAGACAGTACCGAAGAAAACTGATCCTTTCCCTTTAGGGGGTCGGGGAACACGAAGGACCCACTCTGCTACGGGGTGGGACGAAGTCGGGGGGGAGGTACTCGACATGGGTGATAGGTTGGTCAGGGTTACCCTACTTGAACCATGGAGTCTACCTTTACGTC'

In [10]:
print(dataset['train'][0]['input_ids'])

[2, 1560, 2129, 311, 1231, 815, 3248, 692, 2754, 2811, 3038, 3947, 3485, 1638, 2441, 1558, 2124, 289, 1143, 461, 1829, 3208, 532, 2113, 248, 980, 3908, 3332, 1027, 4093, 4070, 3980, 3618, 2171, 477, 1896, 3473, 1591, 2253, 808, 3218, 569, 2263, 847, 3376, 1201, 693, 2760, 2833, 3125, 197, 773, 3079, 14, 44, 161, 630, 2507, 1823, 3182, 426, 1690, 2651, 2399, 1391, 1454, 1706, 2714, 2649, 2392, 1364, 1348, 1284, 1028, 4098, 4091, 4064, 3956, 3524, 1796, 3073, 4085, 4039, 3853, 3111, 144, 561, 2229, 712, 2836, 3137, 247, 975, 3887, 3245, 679, 2702, 2603, 2206, 620, 2467, 1662, 2537, 1943, 3664, 2356, 1220, 772, 3074, 4092, 4068, 3972, 3585, 2039, 4048, 3889, 3253, 712, 2834, 3131, 224, 884, 3524, 1796, 3076, 4100, 4100, 4097, 4088, 4052, 3906, 3321, 983, 3918, 3371, 1184, 625, 2487, 1741, 2854, 3212, 548, 2180, 514, 2044, 4065, 3958, 3529, 1816, 3156, 322, 1274, 988, 3940, 3458, 1531, 2013, 3944, 3476, 1604, 2306, 1018, 4057, 3927, 3407, 1327, 1198, 681, 2711, 2638, 2346, 1180, 609, 2421,

## Train the model:

In [18]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import numpy as np

BATCH_SIZE = 32
LEARNING_RATE = 1e-5
EPOCHS = 3

training_arguments = TrainingArguments(
    output_dir=TMP_DIR, 
    overwrite_output_dir=True,
    evaluation_strategy="epoch", 
    per_device_train_batch_size=BATCH_SIZE, 
    per_device_eval_batch_size=BATCH_SIZE,
    fp16=True,
    save_total_limit=1,
    push_to_hub=True,
    hub_model_id=HF_MODEL_NAME,
    hub_strategy="every_save",
    learning_rate=LEARNING_RATE, 
    num_train_epochs=EPOCHS,
    logging_strategy='epoch',
    )

def compute_metrics(eval_preds):
    metric = load_metric("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
)
trainer

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/roa7n/DNABert_K6_G_quad_3 into local empty directory.
Using cuda_amp half precision backend


In [19]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: seq. If seq are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jovyan/my-conda-envs/ml_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 300000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 28125
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still

Epoch,Training Loss,Validation Loss,Accuracy
1,0.091200,0.088296,0.970700
2,0.066800,0.072255,0.975680
3,0.059800,0.072156,0.976140


Saving model checkpoint to ./q_quad_dnabert/checkpoint-500
Configuration saved in ./q_quad_dnabert/checkpoint-500/config.json
Model weights saved in ./q_quad_dnabert/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./q_quad_dnabert/checkpoint-1000
Configuration saved in ./q_quad_dnabert/checkpoint-1000/config.json
Model weights saved in ./q_quad_dnabert/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [q_quad_dnabert/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./q_quad_dnabert/checkpoint-1500
Configuration saved in ./q_quad_dnabert/checkpoint-1500/config.json
Model weights saved in ./q_quad_dnabert/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [q_quad_dnabert/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to ./q_quad_dnabert/checkpoint-2000
Configuration saved in ./q_quad_dnabert/checkpoint-2000/config.json
Model weights saved in ./q_quad_dnabert/checkpoint-2000/pytorch_model.bin
Deleting older check

TrainOutput(global_step=28125, training_loss=0.07260573784722223, metrics={'train_runtime': 5439.5279, 'train_samples_per_second': 165.456, 'train_steps_per_second': 5.17, 'total_flos': 9.1112480694e+16, 'train_loss': 0.07260573784722223, 'epoch': 3.0})

In [20]:
trainer.push_to_hub(HF_MODEL_NAME)

Saving model checkpoint to ./q_quad_dnabert
Configuration saved in ./q_quad_dnabert/config.json
Model weights saved in ./q_quad_dnabert/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/340M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/roa7n/DNABert_K6_G_quad_3
   e7dce62..13acbdf  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.97614}]}
To https://huggingface.co/roa7n/DNABert_K6_G_quad_3
   13acbdf..8bf4b36  main -> main



'https://huggingface.co/roa7n/DNABert_K6_G_quad_3/commit/13acbdfe7a95eeafee69dcd87b9cd31070b8942b'